# Analyze fine-tuning results
This notebook is used to analyze the fine-tuning and the baseline results obtained wiht Grounding DINO.

### 0. Import libraries and load data

In [ ]:
import json
import polars as pl
import plotly.express as px

RESULTS_PATH = "../../experiments/fine_tuning/"
RESULTS_FOLDER_NAME = "logs_test/"

COLORS = ["#cd968e", "#acb0e0", "#aecbdc", "#bcd5c3", "#bfbfbf"]

In [ ]:
with open(f"{RESULTS_PATH}{RESULTS_FOLDER_NAME}evaluation.json", "r") as f:
    evaluation = json.load(f)

with open(f"{RESULTS_PATH}{RESULTS_FOLDER_NAME}logs.json", "r") as f:
    logs = json.load(f)

### 1. Prepare data and display the statistics

In [ ]:
epochs_info = (
    pl.from_dicts(logs)
    .select(
        "epoch_time", "train_loss", "train_map_50", "train_map_50_95", "val_map_50", "val_map_50_95"
    )
    .with_row_index(name="index", offset=1)
    .with_columns(
        pl.col("epoch_time")
        .str.strptime(pl.Time, format="%H:%M:%S")
        .dt.second()
        .alias("epoch_duration (s)")
    )
    .drop("epoch_time")
    .rename(
        {
            "index": "epoch",
            "train_loss": "train loss",
            "train_map_50": "train mAP@0.5",
            "train_map_50_95": "train mAP@0.5-0.95",
            "val_map_50": "val mAP@0.5",
            "val_map_50_95": "val mAP@0.5-0.95",
        }
    )
)
epochs_info

In [ ]:
fig = px.line(
    epochs_info,
    x="epoch",
    y="train loss",
    title="Evolution of the train loss",
    markers=True,
    color_discrete_sequence=COLORS[0:1],
)
fig.show()

In [ ]:
fig = px.line(
    epochs_info.rename({"train mAP@0.5": "train", "val mAP@0.5": "val"}),
    x="epoch",
    y=["train", "val"],
    labels={"value": "mAP@0.5", "variable": "data set"},
    title="Evolution of mAP@0.5",
    markers=True,
    color_discrete_sequence=COLORS,
)
fig.show()

In [ ]:
fig = px.line(
    epochs_info.rename({"train mAP@0.5-0.95": "train", "val mAP@0.5-0.95": "val"}),
    x="epoch",
    y=["train", "val"],
    labels={"value": "mAP@0.5-0.95", "variable": "data set"},
    title="Evolution of mAP@0.5-0.95",
    markers=True,
    color_discrete_sequence=COLORS,
)
fig.show()

In [ ]:
train_val_metrics = epochs_info[-1].drop("epoch", "train loss", "epoch_duration (s)").to_dicts()[0]
for metric_name, metric_value in train_val_metrics.items():
    print(f"{metric_name}: {metric_value}")
print(f"test mAP@50: {evaluation['test_map_50']}\ntest mAP@50-95: {evaluation['test_map_50_95']}")
print(f"training duration: {epochs_info['epoch_duration (s)'].sum()}s")